In [30]:
from datetime import date
import pandas as pd

In [59]:
class Reed :
   colors = {
      "R" : "Red",
      "B" : "Blue",
      "PU" : "Purple",
      "PI" : "Pink",
      "HAL" : "Halloween",
      "BER" : "Berry",
      "MIN" : "Mint"
   }
   staples = {
      "EB" : "E-Brass",
      "OB" : "O-Brass",
      "HC" : "Hard Cork",
      "SC" : "Soft Cork"
   }
   
   def __init__ (self,      color , staple,    len_init_mm,   year,mo,day):
      """ 
      ======================================================
      INPUT
         >>> color      from :
                           "R" Red  "B" Blue "PU" Purple
                           "PI" Pink    "HAL" Halloween
                           "BER" Berry  "MIN" Mint
         >>> staple     from :
                           "OB" O-Brass   "EB" E-Brass
                           "SC" Soft Cork "HC" Hard Cork
         >>> initial Len      float
         >>> date    --> day, month, year
         
      ======================================================
      """
      self.color   = Reed.colors[color]
      self.staple   = Reed.staples[staple]

      self.datemade     =  date(year,mo,day)   
      self.lenInitial   =  len_init_mm
      self.pitchscores  =  []
      self.respscores   =  []
      self.resoscores   =  []
      
      self.isAlive = True
      
   @property
   def pitch (self):
      return sum(self.pitchscores)/len(self.pitchscores)
   @property
   def response (self):
      return sum(self.respscores)/len(self.respscores)
   @property
   def resonance (self):
      return sum(self.resoscores)/len(self.resoscores)
   @property
   def lifespan (self):
      if self.isAlive == True:
         return (date.today() - self.datemade).days
      
   def __repr__ (self):
      c = self.color
      li = self.lenInitial
      d = f'{self.datemade.month}/{self.datemade.day}'
      return f'[[ REED {d} | {c} | ({li}mm) ]]'
   
   def rec (self, pitch, resp, reso):
      self.pitchscores.append(pitch)
      self.respscores.append(resp)
      self.resoscores.append(reso)
      scores = [ round(sum(x) / len(x),1) for x in [self.pitchscores, self.respscores, self.resoscores]]
      return scores
   
   @property
   def asdict (self):
      return {
         "name" : f'Reed {self.datemade.month}/{self.datemade.day}',
         "color" : self.color ,
         "len_init" : self.lenInitial,
         "date made" : self.datemade,
         "life" : self.lifespan
      }
   
class ReedCase :
   def __init__ (self, caseName):
      self.name = caseName
      self.reeds = []
   
   @property
   def df (self):
      return pd.DataFrame([r.asdict for r in self.reeds])
      
   def newReed (self,      color , staple,    len_init_mm,   year,mo,day):
      """ 
      ======================================================
      INPUT
         >>> color      from :
                           "R" Red  "B" Blue "PU" Purple
                           "PI" Pink    "HAL" Halloween
                           "BER" Berry  "MIN" Mint
         >>> staple     from :
                           "OB" O-Brass   "EB" E-Brass
                           "SC" Soft Cork "HC" Hard Cork
         >>> initial Len      float
         >>> date    --> day, month, year
         
      ======================================================
      """
      reed = Reed (color, staple, len_init_mm, year,mo,day)
      self.reeds.append(reed)
      return reed

In [60]:
JTS = ReedCase("Josh's Reeds")

JTS.newReed("PI","OB",72.3,2022,5,4)
JTS.newReed("MIN","SC",72,2022,5,4)
JTS.newReed("B","EB",71.8,2022,5,7)

round(JTS.df.describe(),1)

,len_init,life
count,3.0,3.0
mean,72.0,6.0
std,0.3,1.7
min,71.8,4.0
25%,71.9,5.5
50%,72.0,7.0
75%,72.2,7.0
max,72.3,7.0


In [65]:
JTS.df.len_init.mean()

72.03333333333335